In [1]:
import tensorflow as tf
import os
from pathlib import Path
from tqdm import tqdm

In [2]:
import pandas as pd
import numpy as np

In [3]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import RMSprop, SGD
from tensorflow.keras import Sequential, models, layers

In [4]:
train_dir = Path("Datasets/dogs-vs-cats/train")
test_dir = Path("Datasets/dogs-vs-cats/test1")

In [5]:
train_df = pd.DataFrame({'image_name':os.listdir(train_dir)})
train_df['label'] =train_df['image_name'].apply(lambda x: x.split('.')[0])
train_df

,image_name,label
0,cat,cat
1,dog,dog


In [6]:
test_df = pd.DataFrame({'image_name':os.listdir(test_dir)})
test_df['label'] =test_df['image_name'].apply(lambda x: x.split('.')[0])
test_df

,image_name,label
0,1.jpg,1
1,10.jpg,10
2,100.jpg,100
3,1000.jpg,1000
4,10000.jpg,10000
...,...,...
12495,9995.jpg,9995
12496,9996.jpg,9996
12497,9997.jpg,9997
12498,9998.jpg,9998


In [7]:
# dog_path_train = os.path.join(train_dir, 'dog')
# os.mkdir(dog_path_train)
# dog_df_train = train_df[train_df.label=='dog']


In [8]:
# for n in tqdm(dog_df_train.image_name):
#     os.rename((os.path.join(train_dir, n)), (os.path.join(dog_path_train, n)))

In [9]:
# cat_path_train = os.path.join(train_dir, 'cat')
# os.mkdir(cat_path_train)
# cat_df_train = train_df[train_df.label=='cat']


In [10]:
# for n in tqdm(cat_df_train.image_name):
#     os.rename((os.path.join(train_dir, n)), (os.path.join(cat_path_train, n)))

In [11]:
train_datagen = ImageDataGenerator(rescale= 1./255, validation_split= 0.3)

In [12]:
train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size= (150, 150),
    class_mode='binary',
    subset='training'
)

Found 17500 images belonging to 2 classes.


In [13]:
validation_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size= (150, 150),
    class_mode= 'binary',
    subset= 'validation'
)

Found 7500 images belonging to 2 classes.


In [14]:
model = Sequential([
    layers.Conv2D(16, (3,3), activation= 'relu', input_shape =(150, 150, 3)),
    layers.MaxPooling2D(2,2),
    layers.Conv2D(32, (3,3), activation= 'relu'),
    layers.MaxPooling2D(2,2),
    layers.Conv2D(64, (3,3), activation= 'relu'),
    layers.MaxPooling2D(2,2),
    layers.Conv2D(64, (3,3), activation= 'relu'),
    layers.MaxPooling2D(2,2),
    layers.Conv2D(64, (3,3), activation= 'relu'),
    layers.MaxPooling2D(2,2),
    layers.Flatten(),
    layers.Dense(512, activation= 'relu'),
    layers.Dense(1,activation= 'sigmoid'),
])

In [15]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 148, 148, 16)      448       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 74, 74, 16)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 72, 72, 32)        4640      
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 36, 36, 32)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 34, 34, 64)        18496     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 17, 17, 64)        0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 15, 15, 64)        3

In [16]:
model.compile(loss= 'binary_crossentropy',
             optimizer = RMSprop(learning_rate= 0.001),
              metrics= ['accuracy']
             )

In [17]:
class mycallback(tf.keras.callbacks.Callback):
    def on_epoch_end(self,epoch,logs={}):
        if(logs.get('val_accuracy')>=0.90):
            self.model.stop_training = True
            
callback = mycallback()

In [ ]:
history = model.fit(
    train_generator,
    steps_per_epoch = train_generator.samples//50,#batch_size, 
    epochs = 30,
    verbose=1,
    validation_data = validation_generator,
    validation_steps = validation_generator.samples//50,#batch_size,
    callbacks=[callback]
)

Epoch 1/30
350/350 [==============================] - 1398s 4s/step - loss: 0.6645 - accuracy: 0.5979 - val_loss: 0.5959 - val_accuracy: 0.6777
Epoch 2/30
350/350 [==============================] - 1314s 4s/step - loss: 0.5648 - accuracy: 0.7081 - val_loss: 0.6283 - val_accuracy: 0.6752
Epoch 3/30
350/350 [==============================] - 1778s 5s/step - loss: 0.5076 - accuracy: 0.7533 - val_loss: 0.4409 - val_accuracy: 0.7958
Epoch 4/30
350/350 [==============================] - 1116s 3s/step - loss: 0.4478 - accuracy: 0.7902 - val_loss: 0.4309 - val_accuracy: 0.7998
Epoch 5/30
350/350 [==============================] - 727s 2s/step - loss: 0.3958 - accuracy: 0.8196 - val_loss: 0.3708 - val_accuracy: 0.8331
Epoch 6/30
350/350 [==============================] - 672s 2s/step - loss: 0.3529 - accuracy: 0.8454 - val_loss: 0.3488 - val_accuracy: 0.8475
Epoch 7/30
350/350 [==============================] - 606s 2s/step - loss: 0.3226 - accuracy: 0.8594 - val_loss: 0.3272 - val_accuracy: 0.

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']

loss = history.history['loss']
val_loss = history.history['val_loss']

plt.figure(figsize=(8, 8))
plt.subplot(2, 1, 1)
plt.plot(acc, label='Training Accuracy')
plt.plot(val_acc, label='Validation Accuracy')
plt.legend(loc='lower right')
plt.ylabel('Accuracy')
plt.ylim([min(plt.ylim()),1])
plt.title('Training and Validation Accuracy')

plt.subplot(2, 1, 2)
plt.plot(loss, label='Training Loss')
plt.plot(val_loss, label='Validation Loss')
plt.legend(loc='upper right')
plt.ylabel('Cross Entropy')
plt.ylim([0,1.0])
plt.title('Training and Validation Loss')
plt.xlabel('epoch')
plt.show()

In [ ]:
 import numpy as np
from tensorflow.keras.preprocessing import image
import os

#new = Path.('\dogs-vs-cats\test1')
imag = os.chdir(r'C:\Users\Hp 15\Documents\My_files\Image Classification\dogs-vs-cats\test1')

images = os.listdir(r'C:\Users\Hp 15\Documents\My_files\Image Classification\dogs-vs-cats\test1')
print(images)



In [ ]:
for i in images:
    print()
    # predicting images
    path = r'C:\Users\Hp 15\Documents\My_files\Image Classification\dogs-vs-cats\test1/' + i
    img = image.load_img(path, target_size=(150, 150))
    x = image.img_to_array(img)
    x /= 255
    x = np.expand_dims(x, axis=0)

    images = np.vstack([x])
    classes = model.predict(images, batch_size=10)
    print(classes[0])
    results = []
    if classes[0]>0.5:
        results.append('Dog')
    else:
        results.append('Cat')

In [ ]:
print(results)

In [ ]:
img = os.listdir(r'C:\Users\Hp 15\Documents\My_files\Image Classification\tmp')
print(img)

In [ ]:
for i in img:
    print()
    # predicting images
    path = r'C:\Users\Hp 15\Documents\My_files\Image Classification\tmp/' + i
    img = image.load_img(path, target_size=(150, 150))
    x = image.img_to_array(img)
    x /= 255
    x = np.expand_dims(x, axis=0)

    images = np.vstack([x])
    classes = model.predict(images, batch_size=10)
    print(classes[0])
    results = []
    if classes[0]>0.5:
        results.append('Dog')
    else:
        results.append('Cat')

In [ ]:
print(results)